# Automatizarea intocmirii fisei pacientului

### Scop

Dezvoltarea unui sistem inteligent care să ajute medicii pentru a completa in mod automat fisa pacientului.

### Ideea de baza

Munca medicilor este plina de provocari. Mai ales cand trebuie sa faca multe task-uri, uneori simultan, precum realizarea si citirea unei ecografii si inregistrarea observatiilor facute. De aceea este nevoie de un sistem inteligent care sa transforme informatia audio inregistrata de catre un medic in format text si sa completeze in mod automat rubricile dedicate din fisa pacientului. Se va pleca de la inregistrari audio, se vor converti in format text si se va completa automat partea evidentiata cu galben din fisa pacientului (informatiile respective se vor salva intr-un tabel/jason si apoi se vor exporta intr-un document word)

---

### Descriere Formală (Matematică / Tehnică)

Datele de intrare:
𝑋 = { fișiere audio_i }
- Explicație: X reprezintă mulțimea fișierelor audio înregistrate de medic, fiecare corespunzând unei observații sau consultații.

Scopul aplicației:
𝐹: 𝑋 → 𝑌
- Explicație: F este funcția care transformă fișierele audio în fișe pacient completate.

Unde:
𝑌 = { fișe pacient completate }
- Explicație: Y este mulțimea fișelor completate cu informații structurate extrase din audio.

---

### Descompunerea Problemei

Problema poate fi descompusă în două sub-probleme:
##### 1. Speech-to-Text
𝑓₁ : audio → text
- Explicație: f1 folosește modele ASR (Romanian Wav2Vec2) pentru a transforma fișierele audio în text.
##### 2. Information Extraction
𝑓₂ : text → date structurate
- Explicație: f2 extrage rubricile relevante din text pentru completarea automată a fișei pacientului.

### Funcția Finală

Funcția finală este:
𝐹 = 𝑓₂ ∘ 𝑓₁
- Explicație: mai întâi transformăm audio-ul în text, apoi extragem datele structurate pentru completarea fișei pacientului.



In [1]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import shutil
import os

In [2]:
import torch
import soundfile as sf
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# --- CONFIG ---
MODEL_ID = "alexvladu1/whisper_finetuned_ro"  # modelul de pe HuggingFace
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- LOAD MODEL & PROCESSOR ---
processor = WhisperProcessor.from_pretrained(MODEL_ID)
model = WhisperForConditionalGeneration.from_pretrained(MODEL_ID).to(DEVICE)
model.eval()

def preprocess_audio(audio_path):
    """Încarcă audio, convertește la mono, resamplează la 16 kHz și pregătește input features."""
    waveform, sr = sf.read(audio_path, dtype="float32")

    # stereo -> mono
    if len(waveform.shape) > 1:
        waveform = waveform.mean(axis=1)

    # resampling
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(sr, 16000)
        waveform = resampler(torch.tensor(waveform)).numpy()
        sr = 16000

    return waveform, sr

def transcribe(audio_path):
    waveform, sr = preprocess_audio(audio_path)

    # durata audio
    duration_sec = waveform.shape[0] / sr
    print(f"Audio duration: {duration_sec:.1f}s")

    # forțare decodor română
    try:
        forced_decoder_ids = processor.get_decoder_prompt_ids(
            language="romanian", task="transcribe"
        )
    except:
        forced_decoder_ids = None

    # --- CHUNKING PENTRU AUDIO LUNG (30 secunde per chunk) ---
    # Whisper funcționează optim cu segmente de ~30 secunde
    CHUNK_DURATION_SEC = 30
    chunk_samples = CHUNK_DURATION_SEC * sr

    segments = []
    start = 0
    total = waveform.shape[0]
    idx = 0

    while start < total:
        end = min(start + chunk_samples, total)
        chunk_duration = (end - start) / sr
        print(f"Chunk {idx}: {chunk_duration:.1f}s (samples {start} → {end})")

        chunk_wave = waveform[start:end]

        # Procesare chunk
        inputs_chunk = processor(chunk_wave, sampling_rate=16000, return_tensors="pt")
        chunk_features = inputs_chunk["input_features"].to(DEVICE)

        # Generare cu parametri optimizați pentru transcripție completă
        gen = model.generate(
            chunk_features,
            max_length=448,  # Lungimea maximă suportată de Whisper
            num_beams=5,
            temperature=0.0,  # Deterministic pentru consistență
            forced_decoder_ids=forced_decoder_ids,
            pad_token_id=processor.tokenizer.pad_token_id or processor.tokenizer.eos_token_id,
            eos_token_id=processor.tokenizer.eos_token_id,
        )

        decoded = processor.batch_decode(gen, skip_special_tokens=True)[0]

        # Curățare text
        decoded = decoded.strip()
        if decoded:
            segments.append(decoded)
            print(f"   → Transcris: {decoded[:100]}..." if len(decoded) > 100 else f"   → Transcris: {decoded}")

        # Trece la următorul chunk (fără overlap pentru a evita duplicarea)
        start = end
        idx += 1

    # Concatenare segmente
    full_text = " ".join(seg for seg in segments if seg)
    print(f"\n✓ Transcripție completă: {len(full_text)} caractere, {len(segments)} segmente")
    return full_text
transcript=transcribe("dataset/train_wav/MIRPR_1.wav")


C:\Users\Alex\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}. If this is not desired, please set these values explicitly.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Audio duration: 57.1s
Chunk 0: 30.0s (samples 0 → 480000)


A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> to see related `.generate()` flags.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> to see related `.generate()` flags.


   → Transcris: Aorta la inel 8 mm, Aorta la sinusuri 14 mm, Aorta ascendentă 12 mm, Atriu Stâng 15 mm, Ventriculul ...
Chunk 1: 27.1s (samples 480000 → 913749)
   → Transcris: Valva aortică Viteză maximă 1,5, Valva pulmonară Viteză maximă 1,3, Valva tricuspidiană Viteză maxim...

✓ Transcripție completă: 546 caractere, 2 segmente


In [3]:
app = FastAPI()
UPLOAD_FOLDER = "uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

In [4]:
@app.post("/upload-audio/")
async def upload_audio(file: UploadFile = File(...)):
    file_path = os.path.join(UPLOAD_FOLDER, file.filename)
    content = await file.read()
    with open(file_path, "wb") as buffer:
        buffer.write(content)
    file_size = len(content)
    transcription = transcribe(file_path)
    return JSONResponse({
        "filename": file.filename,
        "size_bytes": file_size,
        "transcription": transcription
    })

In [5]:
import json
def evaluate_transcription(reference, transcript):
    """
    Evaluează calitatea transcrierii prin multiple metrici de eroare.

    Args:
        reference: Lista sau string cu textul de referință
        transcript: Lista sau string cu textul transcris

    Returns:
        dict: Dicționar cu diverse metrici de eroare
    """
    import re
    from difflib import SequenceMatcher
    from collections import Counter

    # Normalizare input
    if isinstance(reference, list):
        reference = " ".join(reference)
    if isinstance(transcript, list):
        transcript = " ".join(transcript)

    # Funcție de normalizare text medical
    def normalize_medical_text(text):
        """Normalizează text medical pentru comparație mai precisă."""
        # Convertire la lowercase
        text = text.lower()

        # Normalizare spații și punctuație
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[,;]', '', text)

        # Normalizare termeni medicali comuni
        replacements = {
            'aertă': 'aortă',
            'aorta': 'aortă',
            'ventricular': 'ventriculară',
            'vd': 'ventricul drept',
            'vs': 'ventricul stâng',
            'siv': 'sept interventricular',
            'ase': 'atriul stâng esofag',
            'vese': 'ventricul stâng end sistolic',
            't-ape': 'tapse',
        }

        for old, new in replacements.items():
            text = re.sub(r'\b' + old + r'\b', new, text)

        return text.strip()

    # Normalizare texte
    ref_normalized = normalize_medical_text(reference)
    trans_normalized = normalize_medical_text(transcript)

    # Split în cuvinte
    ref_words = ref_normalized.split()
    trans_words = trans_normalized.split()

    # Split în caractere (pentru CER)
    ref_chars = list(ref_normalized.replace(' ', ''))
    trans_chars = list(trans_normalized.replace(' ', ''))

    # === 1. Word Error Rate (WER) ===
    def levenshtein_distance(seq1, seq2):
        """Calculează distanța Levenshtein între două secvențe."""
        size_x = len(seq1) + 1
        size_y = len(seq2) + 1
        matrix = [[0] * size_y for _ in range(size_x)]

        for x in range(size_x):
            matrix[x][0] = x
        for y in range(size_y):
            matrix[0][y] = y

        for x in range(1, size_x):
            for y in range(1, size_y):
                if seq1[x-1] == seq2[y-1]:
                    matrix[x][y] = matrix[x-1][y-1]
                else:
                    matrix[x][y] = min(
                        matrix[x-1][y] + 1,      # deletion
                        matrix[x][y-1] + 1,      # insertion
                        matrix[x-1][y-1] + 1     # substitution
                    )

        return matrix[size_x - 1][size_y - 1]

    # Calcul WER
    word_distance = levenshtein_distance(ref_words, trans_words)
    wer = (word_distance / len(ref_words)) * 100 if ref_words else 0

    # === 2. Character Error Rate (CER) ===
    char_distance = levenshtein_distance(ref_chars, trans_chars)
    cer = (char_distance / len(ref_chars)) * 100 if ref_chars else 0

    # === 3. Similarity Score (SequenceMatcher) ===
    similarity = SequenceMatcher(None, ref_normalized, trans_normalized).ratio() * 100

    # === 4. Extragere valori numerice medicale ===
    def extract_medical_values(text):
        """Extrage valori numerice și unitățile lor din text medical."""
        # Pattern pentru numere cu unități opționale
        pattern = r'(\d+(?:[.,]\d+)?)\s*(?:mm|cm|ml|%|pe)?'
        matches = re.findall(pattern, text)
        return [float(m.replace(',', '.')) for m in matches]

    ref_values = extract_medical_values(reference)
    trans_values = extract_medical_values(transcript)

    # Calcul eroare pentru valori numerice
    value_errors = []
    if len(ref_values) == len(trans_values):
        for rv, tv in zip(ref_values, trans_values):
            if rv != 0:
                error = abs(rv - tv) / rv * 100
                value_errors.append(error)

    avg_value_error = sum(value_errors) / len(value_errors) if value_errors else None

    # === 5. Termeni medicali critici ===
    critical_terms = [
        'aortă', 'ventricul', 'atriul', 'valvă', 'sept',
        'fracție de ejecție', 'restrictiv', 'diametru'
    ]

    ref_critical = [term for term in critical_terms if term in ref_normalized]
    trans_critical = [term for term in critical_terms if term in trans_normalized]

    critical_term_accuracy = (
        len(set(ref_critical) & set(trans_critical)) / len(ref_critical) * 100
        if ref_critical else 100
    )

    # === 6. Analiza diferențelor ===
    matcher = SequenceMatcher(None, ref_words, trans_words)
    operations = {
        'substitutions': 0,
        'insertions': 0,
        'deletions': 0
    }

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == 'replace':
            operations['substitutions'] += max(i2 - i1, j2 - j1)
        elif tag == 'insert':
            operations['insertions'] += j2 - j1
        elif tag == 'delete':
            operations['deletions'] += i2 - i1

    # === Rezultate finale ===
    results = {
        'WER (Word Error Rate)': round(wer, 2),
        'CER (Character Error Rate)': round(cer, 2),
        'Similarity Score': round(similarity, 2),
        'Critical Terms Accuracy': round(critical_term_accuracy, 2),
        'Operations': operations,
        'Word Count': {
            'reference': len(ref_words),
            'transcript': len(trans_words),
            'difference': len(trans_words) - len(ref_words)
        },
    }

    return results
reference = ["Aorta la inel 8 mm, Aorta la sinusuri 14 mm, Aorta ascendentă 12 mm, Atriu Stâng 15 mm, Ventricul Drept 7 mm, Sept Interventricular 5 mm, Ventricul Stâng 24 mm pe 14 mm, Perete posterior 6 mm, Fracție de Injecție 63%, Timp de Accelerare Pulmonară/Măsuri Arteră Pulmonară 16-9-9, Valva Aortică Viteza maxima 1,5, Valva Pulmonară Viteza maxima 1,3, Valva Tricuspidiană Viteza maxima 2,6, Presiune Sistolică în Ventriculul Drept 29 + 5, Arca Aortică Viteza maxima 1,6, cu Persistență de Canal Arterial mic restrictiv cu diametru de aproximativ 1,1 mm."]
results = evaluate_transcription(reference, transcript)
print(transcript)
print(json.dumps(results, indent=2))

Aorta la inel 8 mm, Aorta la sinusuri 14 mm, Aorta ascendentă 12 mm, Atriu Stâng 15 mm, Ventriculul Drept 7 mm, Sept Interventricular 5 mm, Ventricul Stâng 24 mm pe 14 mm, Perete Posterior 6 mm, Fracție de Injecție 63%, Timp de Accelerare Pulmonară Măsuri Arteră Pulmonară 16 mm. Valva aortică Viteză maximă 1,5, Valva pulmonară Viteză maximă 1,3, Valva tricuspidiană Viteză maximă 2,6, presiunea sistolică înventricul drept 29,5, Arca Aortică Viteză maximă 1,6 cu persistență de canal Arterial mic restrictiv cu diametru de aproximativ 1,1,1 mm.
{
  "WER (Word Error Rate)": 22.47,
  "CER (Character Error Rate)": 4.55,
  "Similarity Score": 95.26,
  "Critical Terms Accuracy": 100.0,
  "Operations": {
    "substitutions": 20,
    "insertions": 0,
    "deletions": 0
  },
  "Word Count": {
    "reference": 89,
    "transcript": 88,
    "difference": -1
  }
}


In [ ]:
from transformers import pipeline
# === EXTRACTIE ENTITATI MEDICALE CU MODEL NER GENERIC (PENTRU COMPARATIE) ===
CALEA_MODEL_NER = "alexvladu1/bert-romanian-medical-issue"

try:
    # Incarcam pipeline-ul NER generic
    print(f"Se incarca modelul NER: {CALEA_MODEL_NER}...")
    ner_pipeline = pipeline(
        "ner",
        model=CALEA_MODEL_NER,
        tokenizer=CALEA_MODEL_NER,
        aggregation_strategy="simple"
    )
    print(" Model incarcat cu succes.")

    # Rulam textul prin pipeline pentru a extrage entitatile
    entitati_extrase = ner_pipeline(transcript)

    print("\n--- Entitati Extrase de NER Generic ---")
    for entitate in entitati_extrase:
        print(f"- {entitate['entity_group']}: {entitate['word']} (scor: {entitate['score']:.2f})")

    # Conert into structured JSON
    fisa_pacient_json_generic = {}
    for entitate in entitati_extrase:
        tip_entitate = entitate['entity_group']
        valoare = entitate['word']
        if tip_entitate not in fisa_pacient_json_generic:
            fisa_pacient_json_generic[tip_entitate] = []
        fisa_pacient_json_generic[tip_entitate].append(valoare)

    print("\n--- JSON Structurat Generat (Model Generic) ---")
    json_output_string = json.dumps(fisa_pacient_json_generic, indent=4, ensure_ascii=False)
    print(json_output_string)

    # Salvare în fișier
    with open("data/fisa_pacient_output_generalist.json", "w", encoding="utf-8") as f:
        f.write(json_output_string)
    print(f"\nFisierul 'data/fisa_pacient_output_generalist.json' a fost salvat.")

except Exception as e:
    print(f"A aparut o eroare la încarcarea modelului sau la procesare: {e}")

Se incarca modelul NER: alexvladu1/bert-romanian-medical-issue...



Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


 Model incarcat cu succes.

--- Entitati Extrase de NER Generic ---
- LABEL_1: Aorta (scor: 0.99)
- LABEL_0: la inel 8 mm, (scor: 0.95)
- LABEL_1: Aorta (scor: 0.99)
- LABEL_0: la (scor: 0.97)
- LABEL_1: sinusuri (scor: 0.56)
- LABEL_0: 14 mm, (scor: 1.00)
- LABEL_1: Aorta (scor: 0.99)
- LABEL_2: ascendentă (scor: 0.94)
- LABEL_0: 12 mm, (scor: 1.00)
- LABEL_1: Atriu (scor: 0.98)
- LABEL_2: Stâng (scor: 0.86)
- LABEL_0: 15 mm, (scor: 1.00)
- LABEL_1: Ventriculul (scor: 0.99)
- LABEL_2: Drept (scor: 0.98)
- LABEL_0: 7 mm, (scor: 1.00)
- LABEL_1: Sept (scor: 0.76)
- LABEL_0: Interventricular 5 mm, (scor: 0.90)
- LABEL_1: Ventricul (scor: 0.97)
- LABEL_2: Stân (scor: 0.78)
- LABEL_0: ##g 24 mm pe 14 mm, (scor: 0.95)
- LABEL_1: Perete (scor: 0.97)
- LABEL_2: Posterior (scor: 0.93)
- LABEL_0: 6 mm, Fracție de Injecție 63 %, Timp de Accelerare Pulmonară Măsuri (scor: 1.00)
- LABEL_1: Arteră (scor: 0.98)
- LABEL_2: Pulmonară (scor: 0.95)
- LABEL_0: 16 mm. (scor: 1.00)
- LABEL_1: Valva (scor: 


MASURUTORI ECOGRAFICE EXTRASE:
--------------------------------------------------------------------------------
1. aorta la inel: 8.0 mm
2. aorta la sinusuri: 14.0 mm
3. aorta ascendentă: 12.0 mm
4. ventricul stâng: 24.0 mm
5. atriu stâng: 15.0 mm
6. sept interventricular: 5.0 mm
7. perete posterior: 6.0 mm

MEDICAMENTE EXTRASE:
--------------------------------------------------------------------------------
   Niciun medicament detectat

SIMPTOME EXTRASE:
--------------------------------------------------------------------------------
   Niciun simptom detectat

DIAGNOSTICE EXTRASE:
--------------------------------------------------------------------------------
   Niciun diagnostic detectat

SALVARE DATE STRUCTURATE
Fisa pacientului salvata in: fisa_pacient_medical_structured.json
Observații FHIR salvate în: fhir_observations.json

JSON COMPLET (cu format FHIR)
{
    "masuratori_ecografice": [
        {
            "structura_anatomica": "aorta la inel",
            "valoare_numeric

In [9]:
from core.word_report_generator import MedicalReportGenerator, generate_word_report
from datetime import datetime

# Metoda 1: Raport simplu (fără șablon)
print("\nGenerare raport simplu...")
try:
    raport_path = generate_word_report(
        json_path="data/fisa_pacient_medical_structured.json",
        output_path=f"data/reports/raport_medical_{datetime.now().strftime('%Y%m%d_%H%M%S')}.docx",
        use_template=False
    )
    print(f"Raport generat: {raport_path}")
except Exception as e:
    print(f"Eroare la generarea raportului: {e}")

# Metoda 2: Raport cu șablon personalizat
print("\nGenerare șablon Word personalizabil...")
generator = MedicalReportGenerator()
try:
    generator.create_template("templates/template_fisa_pacient.docx")

    # Generează raport folosind șablonul
    print("\nGenerare raport din șablon...")
    raport_template_path = generate_word_report(
        json_path="data/fisa_pacient_medical_structured.json",
        output_path=f"data/reports/raport_medical.docx",
        use_template=True,
        template_path="templates/template_fisa_pacient.docx"
    )
    print(f"Raport din șablon generat: {raport_template_path}")
except Exception as e:
    print(f"Eroare la generarea raportului cu șablon: {e}")

print("\nREZULTATE FINALE:")
print("   Audio → Text (ASR cu Whisper)")
print("   Text → Entități Structurate (Pattern Matching Medical)")
print("   Entități → JSON (cu format FHIR)")
print("   JSON → Raport Word (Formatat și Lizibil)")
print("\nURMĂTORII PAȘI:")
print("   1. Editează templates/template_fisa_pacient.docx în Word pentru personalizare")
print("   2. Fine-tuning model NER pentru domeniul medical (opțional, pentru acuratețe maximă)")
print("   3. Integrează cu sistemul medical existent (FHIR, HL7)")


C:\Users\Alex\AppData\Local\Programs\Python\Python310\lib\site-packages\docxcompose\properties.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources



Generare raport simplu...
Raport Word generat cu succes: raport_medical_20251206_212650.docx
Raport generat: raport_medical_20251206_212650.docx

Generare șablon Word personalizabil...
sablon creat: template_fisa_pacient.docx
Poți edita acest sablon in Word si apoi il poți folosi cu create_report_with_template()

Generare raport din șablon...
Raport Word generat din sablon: storage/raport_medical.docx
Raport din șablon generat: storage/raport_medical.docx

REZULTATE FINALE:
   Audio → Text (ASR cu Whisper)
   Text → Entități Structurate (Pattern Matching Medical)
   Entități → JSON (cu format FHIR)
   JSON → Raport Word (Formatat și Lizibil)

URMĂTORII PAȘI:
   1. Editează template_fisa_pacient.docx în Word pentru personalizare
   2. Fine-tuning model NER pentru domeniul medical (opțional, pentru acuratețe maximă)
   3. Integrează cu sistemul medical existent (FHIR, HL7)


In [10]:
from fastapi.responses import FileResponse

@app.get("/download-docx")
def download_file():
    FILE_PATH="storage/raport_medical.docx"
    if os.path.exists(FILE_PATH):
        return FileResponse(FILE_PATH, media_type='application/vnd.openxmlformats-officedocument.wordprocessingml.document', filename="raport_medical.docx")
    return {"error": "File not found"}

@app.get("/download-txt")
def download_file():
    return transcript

In [11]:
import nest_asyncio
import uvicorn
import threading

nest_asyncio.apply()

def run_server():
    uvicorn.run(app, host="127.0.0.1", port=8000)

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

print("FastAPI server is running in the background on http://127.0.0.1:8000")

FastAPI server is running in the background on http://127.0.0.1:8000


INFO:     Started server process [1676]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
